<a href="https://colab.research.google.com/github/450586509/Knowledge/blob/master/event_bert_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch
import transformers as tfs
import warnings
import torch.utils.data as Data
from sklearn import preprocessing

warnings.filterwarnings('ignore')

In [2]:
dev_url = "http://129.204.205.246/event_entity_dev_data.csv"
train_url = "http://129.204.205.246/event_entity_train_data_label.csv"
train_df = pd.read_csv(train_url, delimiter='\t', header=None)
train_df.columns = ["id","text","event_name", "company_name"]

KeyboardInterrupt: ignored

In [4]:
print(train_df.shape)
train_df["len"] = train_df.text.apply((lambda x: len(x)))
print(train_df.shape)
print(train_df.head)
for index, row in train_df.iterrows():
  print(f"""index={index} text={row.get("text")} len={len(row.get("text"))}""")
  if index > 5:
    break

(66761, 4)
(66761, 5)
<bound method NDFrame.head of             id  ...  len
0      2444634  ...   48
1      2836026  ...   60
2      2809128  ...   86
3      2221860  ...   62
4      2091205  ...   57
...        ...  ...  ...
66756  2568107  ...   63
66757  2688414  ...   53
66758  2415192  ...  349
66759  2983848  ...  322
66760  2834817  ...   98

[66761 rows x 5 columns]>
index=0 text=世联君汇预计2017年净利下滑近8成至853万元中超电缆(002471)再遭中超集团减持5%股份 len=48
index=1 text=LG空调亏损严重或效仿新科 两大缺陷遭退市尴尬华兰生物(002007)三季度净利下降45% 汇添富或为“失血门”跑路主力 len=60
index=2 text=四方达(300179)股东减持60万股 套现414.6万元收到欧盟打款后希腊总理宣布辞职再选 欧盟紧急声明要求希腊恪守承诺巨力索具(002342)下调预测 因产品毛利率下降 len=86
index=3 text=单元式空调抽查不合格名单春兰、日立产品入列北大荒(600598)计提坏账净利骤降83.5%好利来(002729)股东高位减持 len=62
index=4 text=深航副总裁闪电辞职 “股权之争”仍迷离(IPO路演)龙大肉食(002726)一季度净利“反常”下滑 路演现场遭质疑 len=57
index=5 text=ST皇台(000995)总经理辞职销售公司总经理接任电连技术的“一年之痒”:高管辞职业绩变脸股价破发逾三成 len=53
index=6 text=国润新材因2017年报扔未披露被提示摘牌风险安德利头顶电商压力业绩下坡 自营模式存隐患中小板三公司发布高管辞职公告 len=57


In [5]:
train_df = train_df[train_df.len < 500]
print(f"""max_len={train_df["len"].max()}""")
print(f"""mean_len={train_df["len"].mean()}""")
print(f"shape={train_df.shape}")
train_df.head()

max_len=499
mean_len=86.30794621767306
shape=(66044, 5)


,id,text,event_name,company_name,len
0,2444634,世联君汇预计2017年净利下滑近8成至853万元中超电缆(002471)再遭中超集团减持5%股份,业绩下滑,世联君汇,48
1,2836026,LG空调亏损严重或效仿新科 两大缺陷遭退市尴尬华兰生物(002007)三季度净利下降45% ...,业绩下滑,华兰生物,60
2,2809128,四方达(300179)股东减持60万股 套现414.6万元收到欧盟打款后希腊总理宣布辞职再选...,业绩下滑,巨力索具,86
3,2221860,单元式空调抽查不合格名单春兰、日立产品入列北大荒(600598)计提坏账净利骤降83.5%好...,业绩下滑,北大荒,62
4,2091205,深航副总裁闪电辞职 “股权之争”仍迷离(IPO路演)龙大肉食(002726)一季度净利“反常...,业绩下滑,龙大肉食,57


In [6]:
event_counts = train_df.event_name.value_counts()
print(f"type of event_counts = {type(event_counts)}")
event_counts
print(f"样本最少为{event_counts.min()}")
print(f"样本最多为{event_counts.max()}")
print(f"样本平均为{round(event_counts.mean(),4)}")

type of event_counts = <class 'pandas.core.series.Series'>
样本最少为87
样本最多为7764
样本平均为1503.2222


In [7]:
tokenizer = tfs.BertTokenizer.from_pretrained('bert-base-chinese')
train_tokenized = train_df.text.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(f"train_tokenized type = {type(train_tokenized)}")
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)



train_tokenized type = <class 'pandas.core.series.Series'>


In [8]:
label = train_df.event_name.tolist()
le = preprocessing.LabelEncoder()
enc = OneHotEncoder()
le.fit(label)
y = le.transform(label)
y = [[i] for i in y]
enc.fit(y)
y = enc.transform(y).toarray()
x = train_df.event_name.tolist()
x = np.array(x)
print(type(y))
print(y[0:10])
print(label[0:10])
print(f"x shape ={x.shape}")
print(f"y shape ={y.shape}")

<class 'numpy.ndarray'>
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
['业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑']
x sh

## bert fine-tuning

In [9]:
MAX_LEN = train_max_len
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 1e-05

In [10]:

_,class_number = y.shape
print(f"class_number={class_number}")
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = tfs.BertModel.from_pretrained('bert-base-chinese')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, class_number)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

class_number=28


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [11]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [17]:
#y=ont_hot
#x:文本数据。
class CustomDataset(Dataset):

    def __init__(self, x, y, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = x
        self.targets = y
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        #comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [18]:
# Creating the dataset and dataloader for the neural network
'''
train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
'''
training_set = CustomDataset(x,y, tokenizer, MAX_LEN)
#testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

In [19]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
#testing_loader = DataLoader(testing_set, **test_params)

In [20]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%50==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [21]:
for epoch in range(EPOCHS):
    train(epoch)

RuntimeError: ignored

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
'''
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
'''